In [1]:
import os
import pprint
import sys

path_petsard = os.path.dirname(os.path.dirname(os.getcwd()))
print(path_petsard)
sys.path.append(path_petsard)

pp = pprint.PrettyPrinter(depth=2)

/home/ec2-user/SageMaker/PETsARD


In [2]:
import tempfile

import yaml

from PETsARD import Executor


load_benchmark = 'bank-marketing-2'
syn_expt_name = 'sdv-ctgan'
proc_expt_names = {
    'sdv-copulagan': 'default-sdv',
    'sdv-ctgan': 'default-sdv',
    'sdv-gaussiancopula': 'default-sdv',
    'sdv-tvae': 'default-sdv',
    'smartnoise-aim': 'default-smartnoise',
    'smartnoise-mst': 'default-smartnoise',
    'smartnoise-pacsynth': 'default-smartnoise',
    'smartnoise-dpctgan': 'default-sdv',
    'smartnoise-patectgan': 'default-sdv',
}
preproc_cfgs = {
    'default-sdv': """
Preprocessor:
    default-sdv:
        method: default
""",
    'default-smartnoise': """
Preprocessor:
    default-smartnoise:
        sequence:
            - missing
            - outlier
            - scaler
            - discretizing
""",
}



syn_template: str = """
    {syn_expt_name}:
        method: {syn_expt_name}"""



final_syn_str: str = ''
syn_str = syn_template.format(
    syn_expt_name=syn_expt_name,
)
final_syn_str += syn_str



yaml_template = """
---
Loader:
    {load_benchmark}:
        filepath: benchmark://{load_benchmark}
{preproc_cfg}
Synthesizer:
{final_syn_str}
"""

preproc_expt_name = proc_expt_names[syn_expt_name]
preproc_cfg = preproc_cfgs[preproc_expt_name]

yaml_text = yaml_template.format(
    load_benchmark=load_benchmark,
    preproc_cfg=preproc_cfg,
    final_syn_str=final_syn_str,
)
# pp.pprint(yaml_text)

# cfg = yaml.safe_load(yaml_text)
# pp.pprint(cfg)
with tempfile.NamedTemporaryFile(mode='w+', delete=False) as tmpfile:
    tmpfile.write(yaml_text)
    tmpfile.flush()
    tmpfile.seek(0)
    yaml_file: str = tmpfile.name


exec = Executor(config=yaml_file)
exec.run()

os.remove(yaml_file)

Now is Loader with bank-marketing-2...
Loader - Benchmarker: file benchmark/bank_additional_full.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Now is Preprocessor with default-sdv...
Now is Synthesizer with sdv-ctgan...
Synthesizer (SDV): Fitting CTGAN.
Synthesizer (SDV): Fitting CTGAN spent 773.5354 sec.
Synthesizer (SDV): Sampling CTGAN # 30360 rows (same as raw) in 1.2727 sec.


In [3]:
from copy import deepcopy

from PETsARD import (
    Loader,
    Metadata,
    Processor,
    Synthesizer,
)
from PETsARD.processor.encoder import EncoderUniform


benchmark = 'bank-marketing-2'
syn_method = 'sdv-ctgan'


load = Loader(
    filepath=f"benchmark://{benchmark}",
)
load.load()

proc = Processor(metadata=load.metadata,)
proc.fit(data=load.data)
preproc_data = proc.transform(data=load.data,)


metadata: Metadata = deepcopy(load.metadata)

if 'encoder' in proc._sequence:
    encoder_cfg: dict = proc.get_config()['encoder']
    for col, encoder in encoder_cfg.items():
        if isinstance(encoder, EncoderUniform):
            metadata.set_col_infer_dtype(col, 'numerical')

syn = Synthesizer(method=syn_method,)
syn.create(data=preproc_data, metadata=metadata.to_sdv())
syn.fit_sample()

Loader - Benchmarker: file benchmark/bank_additional_full.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
Synthesizer (SDV): Fitting CTGAN.
Synthesizer (SDV): Fitting CTGAN spent 764.6152 sec.
Synthesizer (SDV): Sampling CTGAN # 30360 rows (same as raw) in 1.237 sec.
